In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Function to load and preprocess data
def preprocess_stock_data(csv_file_path):
    # Step 1: Load dataset
    df = pd.read_csv(csv_file_path)

    # Step 2: Handle missing values
    imputer = SimpleImputer(strategy='median')
    col_miss = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    df[col_miss] = imputer.fit_transform(df[col_miss])

    # Step 3: Detect and remove outliers
    def detect_remove_outliers(df, cols):
        iso = IsolationForest(contamination=0.05, random_state=42)
        for col in cols:
            df['Outlier'] = iso.fit_predict(df[[col]])
            df = df[df['Outlier'] != -1]
        df.drop(columns=['Outlier'], inplace=True)
        return df

    cols_to_check = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    df_cleaned = detect_remove_outliers(df, cols_to_check)

    # Step 4: Target Encoding for 'Symbol' column
    encoder = ce.TargetEncoder(cols=['Symbol'])
    target_col = 'Close'
    df_cleaned['Symbol'] = encoder.fit_transform(df_cleaned['Symbol'], df_cleaned[target_col])

    # Step 5: Add technical indicators
    df_cleaned['SMA_10'] = df_cleaned['Adj Close'].rolling(window=10).mean()
    df_cleaned['SMA_50'] = df_cleaned['Adj Close'].rolling(window=50).mean()
    df_cleaned['EMA_10'] = df_cleaned['Adj Close'].ewm(span=10, adjust=False).mean()
    df_cleaned['EMA_50'] = df_cleaned['Adj Close'].ewm(span=50, adjust=False).mean()
    df_cleaned['SMA_20'] = df_cleaned['Adj Close'].rolling(window=20).mean()
    df_cleaned['STD_20'] = df_cleaned['Adj Close'].rolling(window=20).std()
    df_cleaned['Upper_Band'] = df_cleaned['SMA_20'] + (df_cleaned['STD_20'] * 2)
    df_cleaned['Lower_Band'] = df_cleaned['SMA_20'] - (df_cleaned['STD_20'] * 2)
    df_cleaned['High-Low'] = df_cleaned['High'] - df_cleaned['Low']
    df_cleaned['High-Close'] = np.abs(df_cleaned['High'] - df_cleaned['Adj Close'].shift(1))
    df_cleaned['Low-Close'] = np.abs(df_cleaned['Low'] - df_cleaned['Adj Close'].shift(1))
    df_cleaned['True_Range'] = df_cleaned[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    df_cleaned['ATR'] = df_cleaned['True_Range'].rolling(window=14).mean()
    delta = df_cleaned['Adj Close'].diff(1)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=14).mean()
    avg_loss = pd.Series(loss).rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df_cleaned['RSI'] = 100 - (100 / (1 + rs))
    df_cleaned['EMA_12'] = df_cleaned['Adj Close'].ewm(span=12, adjust=False).mean()
    df_cleaned['EMA_26'] = df_cleaned['Adj Close'].ewm(span=26, adjust=False).mean()
    df_cleaned['MACD'] = df_cleaned['EMA_12'] - df_cleaned['EMA_26']
    df_cleaned['Signal_Line'] = df_cleaned['MACD'].ewm(span=9, adjust=False).mean()
    df_cleaned['VWAP'] = (df_cleaned['Close'] * df_cleaned['Volume']).cumsum() / df_cleaned['Volume'].cumsum()
    df_cleaned['Adj Close_Lag1'] = df_cleaned['Adj Close'].shift(1)
    df_cleaned['Adj Close_Lag2'] = df_cleaned['Adj Close'].shift(2)
    df_cleaned['Adj Close_Lag3'] = df_cleaned['Adj Close'].shift(3)

    # Step 6: Fill NaN values with bfill and ffill
    df_cleaned.fillna(method='bfill', inplace=True)
    df_cleaned.fillna(method='ffill', inplace=True)

    # Step 7: Normalization and Standardization
    numeric_columns = [
        'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol', 
        'SMA_10', 'SMA_50', 'EMA_10', 'EMA_50', 'SMA_20', 'STD_20', 
        'Upper_Band', 'Lower_Band', 'High-Low', 'High-Close', 'Low-Close', 
        'True_Range', 'ATR', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 
        'Signal_Line', 'VWAP', 'Adj Close_Lag1', 'Adj Close_Lag2', 'Adj Close_Lag3'
    ]

    # Normalization
    scaler_minmax = MinMaxScaler()
    df_normalized = df_cleaned.copy()
    df_normalized[numeric_columns] = scaler_minmax.fit_transform(df_cleaned[numeric_columns])

    # Standardization
    scaler_standard = StandardScaler()
    df_standardized = df_cleaned.copy()
    df_standardized[numeric_columns] = scaler_standard.fit_transform(df_cleaned[numeric_columns])

    # Return the cleaned, normalized, and standardized datasets
    return df_cleaned, df_normalized, df_standardized

# Call the function with your CSV file path
df_cleaned, df_normalized, df_standardized = preprocess_stock_data("stocks_sample.csv")


C:\Users\sahoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\Users\sahoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\Windows\Temp\ipykernel_12336\2648518668.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Outlier'] = iso.fit_predict(df[[col]])
C:\Users\sahoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\Users\sahoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarnin

In [2]:
df_normalized.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA_10,...,ATR,RSI,EMA_12,EMA_26,MACD,Signal_Line,VWAP,Adj Close_Lag1,Adj Close_Lag2,Adj Close_Lag3
0,0,2023-09-07,0.093152,0.088376,0.113267,0.110833,0.127507,0.009985,0.188921,0.126061,...,0.012757,0.401345,0.129362,0.131983,0.493794,0.493344,0.013841,0.127507,0.127507,0.127507
1,1,2023-09-08,0.093804,0.088529,0.111333,0.108205,0.124484,0.007932,0.188921,0.126061,...,0.012757,0.401345,0.128890,0.131751,0.493325,0.493238,0.009466,0.127507,0.127507,0.127507
2,2,2023-09-11,0.092663,0.088580,0.113533,0.110769,0.127434,0.013176,0.188921,0.126061,...,0.012757,0.401345,0.128951,0.131762,0.493421,0.493174,0.011218,0.124484,0.127507,0.127507
3,3,2023-09-12,0.093098,0.090363,0.114000,0.112628,0.129572,0.012355,0.188921,0.126061,...,0.012757,0.401345,0.129337,0.131937,0.493831,0.493216,0.013883,0.127434,0.124484,0.127507
4,4,2023-09-13,0.094946,0.089344,0.109333,0.105385,0.121239,0.014637,0.188921,0.126061,...,0.012757,0.401345,0.128362,0.131459,0.492860,0.493029,0.008709,0.129572,0.127434,0.124484


In [3]:
df_standardized.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA_10,...,ATR,RSI,EMA_12,EMA_26,MACD,Signal_Line,VWAP,Adj Close_Lag1,Adj Close_Lag2,Adj Close_Lag3
0,0,2023-09-07,-0.537395,-0.539431,-0.533429,-0.532564,-0.528260,-0.646849,-0.346151,-0.545798,...,-0.428409,-0.635839,-0.539213,-0.550878,-0.000568,-0.000591,-3.226797,-0.528247,-0.528236,-0.528225
1,1,2023-09-08,-0.533678,-0.538514,-0.542536,-0.545275,-0.541037,-0.657474,-0.346151,-0.545798,...,-0.428409,-0.635839,-0.541220,-0.551865,-0.008915,-0.002376,-3.265126,-0.528247,-0.528236,-0.528225
2,2,2023-09-11,-0.540183,-0.538208,-0.532173,-0.532874,-0.528571,-0.630337,-0.346151,-0.545798,...,-0.428409,-0.635839,-0.540960,-0.551816,-0.007216,-0.003441,-3.249779,-0.541025,-0.528236,-0.528225
3,3,2023-09-12,-0.537705,-0.527507,-0.529974,-0.523884,-0.519534,-0.634587,-0.346151,-0.545798,...,-0.428409,-0.635839,-0.539321,-0.551072,0.000095,-0.002729,-3.226430,-0.528559,-0.541014,-0.528225
4,4,2023-09-13,-0.527172,-0.533622,-0.551957,-0.558915,-0.554750,-0.622778,-0.346151,-0.545798,...,-0.428409,-0.635839,-0.543464,-0.553105,-0.017190,-0.005857,-3.271753,-0.519521,-0.528548,-0.541003


In [5]:
df_cleaned.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,SMA_10,...,ATR,RSI,EMA_12,EMA_26,MACD,Signal_Line,VWAP,Adj Close_Lag1,Adj Close_Lag2,Adj Close_Lag3
0,0,2023-09-07,17.139999,17.350000,16.990000,17.290001,17.290001,445000.0,24.530476,16.939,...,0.564286,40.13452,17.290001,17.290001,0.000000,0.000000,17.290001,17.290001,17.290001,17.290001
1,1,2023-09-08,17.260000,17.379999,16.700001,16.879999,16.879999,353500.0,24.530476,16.939,...,0.564286,40.13452,17.226924,17.259630,-0.032707,-0.006541,17.108491,17.290001,17.290001,17.290001
2,2,2023-09-11,17.049999,17.389999,17.030001,17.280001,17.280001,587200.0,24.530476,16.939,...,0.564286,40.13452,17.235089,17.261139,-0.026050,-0.010443,17.181170,16.879999,17.290001,17.290001
3,3,2023-09-12,17.129999,17.740000,17.100000,17.570000,17.570000,550600.0,24.530476,16.939,...,0.564286,40.13452,17.286614,17.284018,0.002596,-0.007835,17.291736,17.280001,16.879999,17.290001
4,4,2023-09-13,17.469999,17.540001,16.400000,16.440001,16.440001,652300.0,24.530476,16.939,...,0.564286,40.13452,17.156366,17.221498,-0.065132,-0.019295,17.077108,17.570000,17.280001,16.879999
